# Kindle Clippings Markdown Exporter

This notebook will generate a separate markdown file for each your books with their clippings and highlightss. 

NOTE: To use, first run [kindle_clippings_parser.ipynb](https://github.com/markwk/qs_ledger/blob/master/kindle/kindle_clippings_parser.ipynb) to parse and collect your device's clippings.

For **data analysis and some data visualization** of your Amazon Kindle clippings, see: [kindle_clippings_data_analysis.ipynb](https://github.com/markwk/qs_ledger/blob/master/kindle/kindle_clippings_data_analysis.ipynb)

----

## Configuration

In [1]:
export_directory = '/Users/my-user-name/path1/path2/path3'

----

## Dependencies

In [2]:
from datetime import date, datetime as dt, timedelta as td
import pandas as pd

----

## Data Import and Data Preparations

In [3]:
my_clippings = pd.read_csv("data/clippings.csv")

In [4]:
my_clippings.columns = ['author', 'book_title', 'timestamp', 'highlight', 'location',
       'num_pages']

In [5]:
# date additions
my_clippings['timestamp'] = pd.to_datetime(my_clippings['timestamp'])
my_clippings['date'] = my_clippings['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d')) # note: not very efficient
my_clippings['year'] = my_clippings['timestamp'].dt.year
my_clippings['month'] = my_clippings['timestamp'].dt.month
my_clippings['mnth_yr'] = my_clippings['timestamp'].apply(lambda x: x.strftime('%Y-%m')) # note: not very efficient
my_clippings['day'] = my_clippings['timestamp'].dt.day
my_clippings['dow'] = my_clippings['timestamp'].dt.weekday
my_clippings['hour'] = my_clippings['timestamp'].dt.hour

----

# Kindle Book Clippings Markdown Exporter

In [6]:
book_titles = my_clippings['book_title'].unique()
print('{:,} total books'.format(len(book_titles)))

463 total books


In [7]:
def generate_book_file(book_title, directory=export_directory):
    book_notes = my_clippings[my_clippings['book_title'] == book_title]
    title = (book_notes.iloc[0]['book_title']).rstrip()
    author = (book_notes.iloc[0]['author']).rstrip()
    title_stripped = (title.rstrip()
                      .replace(" ", "_")
                      .replace(":", "")
                      .replace(",", "")
                      .replace("/", "")
                      .replace("(", "")
                      .replace(")", "")
                      .replace("?", "")
                      .lower())
    filename=(book_notes.iloc[0]['timestamp'].strftime('%Y%m%d%H%M') + "_" + title_stripped+".md")
    filepath= directory+filename
    
    if author == 'Blinkist':
        pass
    else: 
        print("Printing... " + filename)
        file = open(filepath,"w") 
        file.write("# " + title + " by " + author + " \n")
        file.write("### Clippings \n")
        file.write("tags: #BookClippings #BookRead \n")
        file.write(" \n") 
        for index, row in book_notes.iterrows():
            file.write(str(row['highlight']) + " \n")
            file.write("p " + str(row['num_pages']) + " | " + row['location'] + " | " + str(row['timestamp']) + " \n")
            file.write(" \n")
        file.close() 

In [2]:
# Get a Test Book Title
 book_titles[-1]

In [9]:
# Test Individual Book Export
# generate_book_file("Buddha's Brain ")

In [1]:
# loop through all books
for i in book_titles:
    generate_book_file(i)